In [33]:
pip install --upgrade openpyxl

In [34]:
import pandas as pd
import numpy as np
import os
import glob
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")


### 1. mycat system to csv using pandas

In [35]:
def procesar_archivos_cat(path_dir, Shovel):

    df = pd.read_excel(path_dir)
    df['Shovel'] = Shovel

    # Rename specific columns
    df.rename(columns={
        df.columns[5]: 'machine_hours',
        df.columns[8]: 'idle_percentage',
        df.columns[9]: 'operating_hours',
        df.columns[10]: 'total_fuel',
        df.columns[11]: 'gal_hour',
        df.columns[12]: 'Shovel'
    }, inplace=True)

    # Filter useful columns
    columnas_utiles = ['Fecha','machine_hours','idle_percentage','operating_hours',
                       'total_fuel','gal_hour','Shovel']
    df = df[columnas_utiles]

    # Save raw copy
    df_raw= df.copy().set_index('Fecha')

    # Clean data
    df.dropna(subset=['machine_hours'], inplace=True)
    df = df[df['machine_hours'] > 0.5]

    df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')
    df.dropna(subset=['Fecha'], inplace=True)

    # Fill gal_hour
    gal_hour_avg = df.groupby('Shovel')['gal_hour'].mean()
    df['gal_hour'] = df.apply(
        lambda row: gal_hour_avg[row['Shovel']] if pd.isna(row['gal_hour']) else row['gal_hour'], axis=1
    )

    # Calculate total_fuel if null
    df['total_fuel'] = df.apply(
        lambda row: row['machine_hours'] * row['gal_hour'] if pd.isna(row['total_fuel']) else row['total_fuel'],
        axis=1
    )

    # Fill idle_percentage and operating_hours
    porcentaje_avg = df.groupby('Shovel')[['idle_percentage', 'operating_hours']].mean()
    df['idle_percentage'] = df['idle_percentage'].fillna(df['Shovel'].map(porcentaje_avg['idle_percentage']))
    df['operating_hours'] = df['operating_hours'].fillna(df['Shovel'].map(porcentaje_avg['operating_hours']))

    df.set_index('Fecha', inplace=True)

    return df, df_raw


In [36]:
path_dir = "/content/Caterpillar_excel.xlsx"
df, df_raw = procesar_archivos_cat(path_dir, "CAT 336-8")

### 2. Save in CSV format

In [37]:
df.to_csv("Shovels_CAT_clean_output.csv",index=True)

In [38]:
df_raw.to_csv("Shovels_CAT_raw_output.csv",index=True)